# Working with Private Datasets

## Install

In [1]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

✅ The installed version of syft==0.8.2b2 matches the requirement >=0.8.2b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="private-datasets-example-domain-1", port=8062, reset=True)

Starting private-datasets-example-domain-1 server on 0.0.0.0:8062


Waiting for server to start Done.


## Setup

For the purpose of this tutorial we are creating a very simple dataset, which is created and owner by the root client

In [4]:
root_client = node.login(email="info@openmined.org", password="changethis")

Logged into private-datasets-example-domain-1 as <info@openmined.org>


In [5]:
import numpy as np

In [6]:
dataset = sy.Dataset(
    name="my dataset",
    asset_list=[
        sy.Asset(
        name="my asset",
        data=np.array([1,2,3]),
        mock=np.array([1,1,1])
    )]
)

In [7]:
root_client.upload_dataset(dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 14.98it/s]

Uploading: my asset


<class 'syft.service.response.SyftSuccess'>: Dataset uploaded to 'private-datasets-example-domain-1'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [8]:
root_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

<class 'syft.service.response.SyftSuccess'>: User 'Jane Doe' successfully registered! To see users, run `[your_client].users`

## Mocks

In [9]:
guest_client = node.client.login(email="jane@caltech.edu", password="abc123")

Logged into private-datasets-example-domain-1 as <jane@caltech.edu>


Lets inspect the datasets from the data scientists perspective

In [10]:
datasets = guest_client.datasets
datasets

Datasets have assets, in our case there is only 1 asset

In [11]:
asset = datasets[0].assets[0]
asset

0
Loading... (need help?)


When you get a refence to an asset as a datascientist using Pysyft, you are almost never getting the real data. Often you will get a mock object instead, which is an object with the same type and characteristics (e.g. list size), but with fake data instead. In Pysyft, you can access the mock objects in 2 ways. The first method is to call `Asset.mock`

In [12]:
mock = asset.mock

As we can see, the mock data is just a a native library type, and not a type created by PySyft

In [13]:
type(mock), mock

(numpy.ndarray, array([1, 1, 1]))

We can use mock objects to write code against the mock data, which we can then pass to a `@syft_function` to execute remotely. E.g.

In [14]:
x = mock + 3
y = x ** 2

In [15]:
@sy.syft_function(input_policy=sy.ExactMatch(inp=asset),
                  output_policy=sy.SingleExecutionExactOutput())
def add_pow(inp):
    x = inp + 3
    y = x ** 2
    return y

Syft function 'add_pow' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.


We wont go deeper into the flow for approving execution of this here, for more see the `syft function` tutorial

## Eager Execution

`@syft_functions` are useful, but have 2 downsides

- not every data owner wants to execute raw python code
- you can only remotely execute the code once you get approval

The second way to access a reference to our asset is via `Asset.pointer`. `Pointers` are objects that point to data on the server, they can contain mock data as well, but they rarely contain the real data. When you use a `Pointer` to do a computation PySyft does the following things

- a) the computation is performed locally on the mock data
- b) the client sends an `Action` to the server, which causes the computation to be performed on the server
- c) we create a new `Pointer` as a result, which contains the locally created mock data and points to the result on the server

We call B and C here side-effects

In [16]:
pointer = asset.pointer

In [17]:
pointer

```python
TwinPointer(Mock)
```
array([1, 1, 1])

In [18]:
pointer.sum()

```python
TwinPointer(Mock)
```
3

So the `.sum` method we just called did a, b and c behind the scenes. This also happens for the so called dunder methods, these are methods that are implicitly called when we call for instance `pointer + 1`. Under the hood `pointer + 1` is syntactic sugar for `pointer.__add__(1)` which allows the Pointer to intercept this call and create the side effects.

In [19]:
pointer2 = pointer + 1
pointer2

```python
TwinPointer(Mock)
```
array([2, 2, 2])

Another thing to notice here, is that to call `__add__` with `1` as an argument, we also need to have `1` on the server. Therefore, when we are passing arguments to methods, Syft is pointerizing them as well as a side effect before the action is executed on the server.

This gives us a pretty complete picture of how we can execute methods on pointers. Sometimes we want to create objects from scratch, not merely as a result of a method. In eager execution land, this means creating a pointer on the server.

In [20]:
pointer3 = guest_client.numpy.array([4,5,6])
pointer3

```python
Pointer
```
array([4, 5, 6])

This also created a pointer. In this case, we can see the real data (not a mock), as we own this data. We can use the `client.lib_path` pattern for both functions and classes. Morover, we can combine it with the original pointer in the same was as before:

In [21]:
pointer3 = guest_client.numpy.add(pointer, pointer3)

For methods, functions and classes, we can use autocomplete. In a jupyter notebook you can do this by typing the method and the opening brackets, and then calling `shift-tab`, e.g. pointer.max().

**step into the `()` and type shift-tab for auto complete**

In [22]:
pointer.max()

```python
TwinPointer(Mock)
```
1

Note that the Same works for `guest_client.api.lib.numpy.some_function`.

When we are done with our computations, we can request the real result of the computation, instead of the mock. We can do this using the `Pointer.request()` method:

In [23]:
pointer3.request(guest_client)

syft.service.request.request.Request

Data owners can now approve this request

In [24]:
root_client = node.login(email="info@openmined.org", password="changethis")

Logged into private-datasets-example-domain-1 as <info@openmined.org>


In [25]:
requests = root_client.requests
requests

In [26]:
requests[0].approve_with_client(root_client)

<class 'syft.service.response.SyftSuccess'>: Request 09d1eb9e4ff142a79721b86617711cf0 changes applied

Which allows the data scientists to download the result

In [27]:
pointer3.get()

array([5, 7, 9])

## Action Service

### Listing the Services

### Autocomplete Service Methods

### Viewing Method Signatures

## Simple Example

## Request the Result